In [1]:
import pandas as pd
import numpy as np 

In [ ]:
Sp = Sp[Sp.Product.str.isin(["Tomatoes", "Chicken", "Beef Meat", ""])]

In [21]:
Sp = pd.read_pickle("data/processed/Spain_opt_diet.pkl")
test_df = Sp.copy()
test_df = test_df.apply(pd.to_numeric)
test_df = test_df.stack().to_frame().reset_index()
test_df = test_df.rename(columns = {"level_1": "af_countries", 0:"val"})
levels = ['id', 'af_countries'] # levels used for the hierarchical chart
color_columns = ['val']
value_column = 'val'

In [42]:
id_n = test_df.index.values
test_df['id'] = test_df.Product + id_n.astype(str)
test_df = test_df[['Product', 'id', 'af_countries', 'val']]
test_df

,Product,id,af_countries,val
0,Apples,Apples0,Ethiopia,5.401381
1,Apples,Apples1,Kenya,2.248050
2,Apples,Apples2,United Republic of Tanzania,1.840723
3,Apples,Apples3,Madagascar,1.673290
4,Apples,Apples4,Zambia,1.292569
...,...,...,...,...
320,Tomatoes,Tomatoes320,Botswana,15.937875
321,Tomatoes,Tomatoes321,Guinea-Bissau,13.056112
322,Tomatoes,Tomatoes322,Eswatini,9.988576
323,Tomatoes,Tomatoes323,Lesotho,3.768383


In [151]:

def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy, 
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'label', 'parent', 'value', 'log_val','color'])
#     df_all_trees = pd.DataFrame(columns=['id', 'label', 'parent', 'value','color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value','log_val', 'color'])
#         df_tree = pd.DataFrame(columns=['id', 'label', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum(numerical_only=True)
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if(i==0):
            df_tree['label'] = dfg['Product'].copy()
        else:
            df_tree['label'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'Total'
        df_tree['log_val'] = np.log(dfg[value_column]+2)
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='Total', parent='', 
                              value=df_tree['value'].sum(),
                              color=df_tree['color'].sum(), 
                              label="Total",
                              log_val=np.log1p(df[value_column].sum())))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees

In [152]:
df_all_trees = build_hierarchical_dataframe(test_df, levels, value_column, color_columns)
df_all_trees

,color,id,label,log_val,parent,value
0,5.401381,Apples0,Apples,2.001667,Ethiopia,5.401381
1,2.248050,Apples1,Apples,1.446460,Kenya,2.248050
2,0.623758,Apples10,Apples,0.964608,Malawi,0.623758
3,0.444809,Apples11,Apples,0.893967,Zimbabwe,0.444809
4,0.311232,Apples12,Apples,0.837780,Angola,0.311232
...,...,...,...,...,...,...
346,4297.340404,Uganda,Uganda,8.366217,Total,4297.340404
347,8103.112233,United Republic of Tanzania,United Republic of Tanzania,9.000250,Total,8103.112233
348,5690.064837,Zambia,Zambia,8.646828,Total,5690.064837
349,1958.109216,Zimbabwe,Zimbabwe,7.580755,Total,1958.109216


In [156]:
df_all_trees = df_all_trees.replace({'United Republic of Tanzania':'Tanzania', 'Central African Republic':'Centr. Afr. Rep.'})

In [164]:
import plotly.graph_objects as go
import plotly.offline as py

fig =go.Figure(go.Sunburst(
    ids=df_all_trees.id,
    labels=df_all_trees.label,
    parents=df_all_trees.parent,
    maxdepth=2,
    values=df_all_trees.log_val,
    customdata = df_all_trees.value,
    hovertemplate='<b>%{label}</b><br><br>Need: %{customdata:.3f} tons',
    name='',
    marker = dict(colorscale="RdBu"),
#     branchvalues='total'
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=20), paper_bgcolor = "rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()
py.plot(fig, filename='docs/_includes/sunburst.html')

'docs/_includes/sunburst.html'

In [25]:
df_all_trees

,color,id,label,parent,real_values,value
0,5.401381,Apples0,Apples,Ethiopia,1.856514,5.000000
1,2.248050,Apples1,Apples,Kenya,1.178055,2.000000
2,0.623758,Apples10,Apples,Malawi,0.484743,1.000000
3,0.444809,Apples11,Apples,Zimbabwe,0.367977,0.000000
4,0.311232,Apples12,Apples,Angola,0.270967,0.000000
...,...,...,...,...,...,...
346,4297.340404,Uganda,ApplesBarleyBeansBeef MeatCherriesChickenCucum...,total,8.365984,4297.000000
347,8103.112233,United Republic of Tanzania,ApplesBarleyBeansBeef MeatCherriesChickenCucum...,total,9.000127,8103.000000
348,5690.064837,Zambia,ApplesBarleyBeansBeef MeatCherriesChickenCucum...,total,8.646653,5690.000000
349,1958.109216,Zimbabwe,ApplesBarleyBeansBeef MeatCherriesChickenCucum...,total,7.580245,1958.000000
